In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from DSS_analyzer_Mariner import Data1D_GAUGE, Data1D_PumpingCurve
datapath = "./"
gauge_data_folder = datapath + "s_well/gauge_data/"
gauge_dataframe = Data1D_GAUGE.Data1D_GAUGE(gauge_data_folder + f"gauge6_data_swell.npz")
# Plot the gauge data
from DSS_analyzer_Mariner import Data1D_PumpingCurve
pumping_curve_filepath = datapath + f"prod/pumping_curve/pumping_curve_stage7.npz"
pumping_curve = Data1D_PumpingCurve.Data1D_PumpingCurve(pumping_curve_filepath)

# Crop the pressure gauge data,
start_time = pumping_curve.get_start_time()
end_time = pumping_curve.get_end_time()
gauge_dataframe.crop(start_time, end_time)

gauge_md_index = np.array([265, 520,  789, 1043, 1312, 1567])
frac_hit_index = np.array([589, 830])

In [3]:
# Interpolate the gauge data for any time point, the function would return the interpolated value
from scipy.interpolate import interp1d
def interpolate_gauge_data(time):
    gauge_data = gauge_dataframe.data
    gauge_taxis = gauge_dataframe.calculate_time()
    gauge_interpolator = interp1d(gauge_taxis * 3600, gauge_data, kind='linear')
    return gauge_interpolator(time)

In [4]:
# Simulator parameters
nx = 10000
# nx = 1800
total_time = 13000
d = 140 * np.ones(nx)
dx = 1
x = np.arange(0, nx*dx, dx)
x -= 4000
u_snapshot = [] # store the snapshot of u
u_current = np.zeros(nx)
t = list([0]) # store the time points
dt = 0
dt_init = 10.0

In [5]:
# Simulator
def build_matrix_phase1(dt):
    alpha = d * dt / dx**2
    
    A = np.zeros((nx, nx))
    
    for i in range(1, nx-1):
        A[i, i-1] = -alpha[i]
        A[i, i] = 1 + 2 * alpha[i]
        A[i, i+1] = -alpha[i]
    
    A[0, 0] = 1
    A[-1, -1] = 1
    A[0, 1] = -1
    A[-1, -2] = -1
    
# Simulator
def build_matrix_phase1(dt):
    alpha = d * dt / dx**2
    
    A = np.zeros((nx, nx))
    
    for i in range(1, nx-1):
        A[i, i-1] = -alpha[i]
        A[i, i] = 1 + 2 * alpha[i]
        A[i, i+1] = -alpha[i]
    
    A[0, 0] = 1
    A[-1, -1] = 1
    A[0, 1] = -1
    A[-1, -2] = -1
    
    # Source term
    phase1_frac_hit_index = int(np.where(x == frac_hit_index[0])[0])
    A[phase1_frac_hit_index, :] = 0
    A[phase1_frac_hit_index, phase1_frac_hit_index] = 1
    
    return A

def build_rhs_phase1(u_current, t):
    rhs = u_current.copy()
    rhs[0] = 0
    rhs[-1] = 0
    # Source term.
    rhs[frac_hit_index[0]] = interpolate_gauge_data(t)
    # print(f"Time: {t}, Source: {interpolate_gauge_data(t)}") # debug info
    return rhs

In [6]:
u_current = gauge_dataframe.data[0] * np.ones_like(u_current) # initial condition
u_snapshot.append(u_current.copy())

In [7]:
# full step
def full_step_phase1(u_current, dt, t):
    A = build_matrix_phase1(dt)
    rhs = build_rhs_phase1(u_current, t)
    return np.linalg.solve(A, rhs)

# half step
def half_step_phase1(u_current, dt, t):
    dt_half = dt / 2
    A_half = build_matrix_phase1(dt_half)
    rhs_half = build_rhs_phase1(u_current, t)
    u_middle = np.linalg.solve(A_half, rhs_half)
    
    b_half = build_rhs_phase1(u_middle, t)
    u_final = np.linalg.solve(A_half, b_half)
    
    return u_final

In [8]:
# Time sampling optimizer
def adjust_dt(dt_current, error, tol=1e-3):
    safety_factor = 0.9
    p = 2
    
    dt_new = safety_factor * dt_current * (tol / max(error, 1e-16))**(1/(p+1))
    
    dt_new = min(2.0 * dt_init, max(0.5 * dt_current, dt_new))
    
    return dt_new

In [9]:
# test the full step
dt = dt_init

u_current = full_step_phase1(u_current, dt, t[-1])
u_snapshot.append(u_current.copy())
t.append(t[-1] + dt)

In [ ]:
while t[-1] < total_time: # time loop
    print("dt = ", dt)
    # calculate the full step
    u_current = full_step_phase1(u_current, dt, t[-1])
    
    # calculate the half step
    u_current_half = half_step_phase1(u_current, dt, t[-1])
    
    # calculate the error
    error = np.linalg.norm(u_current_half - u_current) / np.linalg.norm(u_current)
    # print(f"Time: {t[-1]}, Error: {error}")
    # adjust the time step
    dt = adjust_dt(dt, error)
    
    if error <= 1e-3:
        # store the snapshot if the error is small enough.
        u_snapshot.append(u_current.copy())
        t.append(t[-1] + dt)
        
        # update the time step
        dt = min(dt, 10)
    else:
        # the error is too large, reduce the time step. Don't store the snapshot.
        dt = max(0.5 * dt, 0.5)
    
    # store the snapshot
    u_snapshot.append(u_current.copy())
    print(f"Time: {t[-1]}, Error: {error:.6f}")
    t.append(t[-1] + dt)

dt =  10.0
Time: 30.0, Error: 0.000000
dt =  10
Time: 60.0, Error: 0.000002
dt =  10
Time: 90.0, Error: 0.000003
dt =  10


In [ ]:
# plot the results
plt.figure(figsize=(10, 6))
plt.imshow(np.array(u_snapshot).T, aspect='auto', cmap='bwr', extent=[0, t[-1], 0, frac_hit_index[-1] + 400])
plt.colorbar()
plt.xlabel("Time (s)")
plt.ylabel("Distance (ft)")
plt.show()

# After stage 1 simulation, before stage 2 simulation. 

In [ ]:
total_time_interval = 3000

In [ ]:
# Simulator
def build_matrix_phase2(dt):
    alpha = d * dt / dx**2
    
    A = np.zeros((nx, nx))
    
    for i in range(1, nx-1):
        A[i, i-1] = -alpha[i]
        A[i, i] = 1 + 2 * alpha[i]
        A[i, i+1] = -alpha[i]
    
    A[0, 0] = 1
    A[-1, -1] = 1
    A[0, 1] = -1
    A[-1, -2] = -1
    
    # Source term: there is no source term in phase 2.
    return A

def build_rhs_phase2(u_current, t):
    rhs = u_current.copy()
    rhs[0] = 0
    rhs[-1] = 0
    # Source term. There is no source term in phase 2.
    return rhs

# full step
def full_step_phase2(u_current, dt, t):
    A = build_matrix_phase2(dt)
    rhs = build_rhs_phase2(u_current, t)
    return np.linalg.solve(A, rhs)

# half step
def half_step_phase2(u_current, dt, t):
    dt_half = dt / 2
    A_half = build_matrix_phase2(dt_half)
    rhs_half = build_rhs_phase2(u_current, t)
    u_middle = np.linalg.solve(A_half, rhs_half)
    
    b_half = build_rhs_phase2(u_middle, t)
    u_final = np.linalg.solve(A_half, b_half)
    
    return u_final

In [ ]:
total_time += total_time_interval # update the total time, now it's stage1 injection + after stage1 + stage2 injection
while t[-1] < total_time: # time loop
    print("dt = ", dt)
    # calculate the full step
    u_current = full_step_phase2(u_current, dt, t[-1])
    
    # calculate the half step
    u_current_half = half_step_phase2(u_current, dt, t[-1])
    
    # calculate the error
    error = np.linalg.norm(u_current_half - u_current) / np.linalg.norm(u_current)
    # print(f"Time: {t[-1]}, Error: {error}")
    # adjust the time step
    dt = adjust_dt(dt, error)
    
    if error <= 1e-3:
        # store the snapshot if the error is small enough.
        u_snapshot.append(u_current.copy())
        t.append(t[-1] + dt)
        
        # update the time step
        dt = min(dt, 10)
    else:
        # the error is too large, reduce the time step. Don't store the snapshot.
        dt = max(0.5 * dt, 0.5)
    
    # store the snapshot
    u_snapshot.append(u_current.copy())
    print(f"Time: {t[-1]}, Error: {error:.6f}")
    t.append(t[-1] + dt)